In [1]:
import subprocess
import os
import pandas as pd
import datetime as dt
import re
import json
from collections import namedtuple

import matplotlib.pyplot as plt

In [28]:
def parse_mi_ho(f):

    df = pd.read_csv(f)
    df["Timestamp"] = df["Timestamp"].swifter.apply(lambda x: pd.to_datetime(x) + dt.timedelta(hours=8)) 
    nr_pci = 'O'
    scells = []

    def NR_OTA(idx):

        if df["type_id"].iloc[idx] == "5G_NR_RRC_OTA_Packet": return True
        else: return False
    
    def LTE_SERV_INFO(idx):

        if df["type_id"].iloc[idx] == "LTE_RRC_Serv_Cell_Info": return True
        else: return False
    

    def find_1st_after(start_idx, target, look_after=1):
        for j in range(start_idx, len(df)):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t_ - t).total_seconds() > look_after:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before(start_idx, target, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] not in [0,'0'] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_1st_before_with_special_value(start_idx, target, target_value, look_before=1):
        for j in range(start_idx, -1, -1):
            t_ = df["Timestamp"].iloc[j]
            if NR_OTA(j) or LTE_SERV_INFO(j):
                continue
            if (t - t_).total_seconds() > look_before:
                return None, None
            if df[target].iloc[j] in [target_value] and not np.isnan(df[target].iloc[j]):
                return t_, j
        return None, None
    
    def find_in_D_exact(targets):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.start).total_seconds(), target))

        if len(l) != 0:
            for x in l:
                if (x[0]== 0):
                    return x[1]
        
        return None
    
    def find_in_D_first_before(targets, look_before=1):

        l = []
        # In l : (second, ho_type)
        for target in targets:
            for ho in D[target]:
                l.append(((t - ho.end).total_seconds(), target, ho))

        if len(l) != 0:
            closest = min(filter(lambda x: x[0] > 0, l), key=lambda x: x[0])
            if 0 <= closest[0] < look_before:
                return closest[1], closest[2]
        
        return None, None

    HO = namedtuple('HO',['start', 'end', 'others', 'trans'], defaults=[None,None,'',''])
    
    D = {
        'Conn_Rel':[], 
        'Conn_Req':[], # Setup
        'LTE_HO': [], # LTE -> newLTE
        'MN_HO': [], # LTE + NR -> newLTE + NR
        'MN_HO_to_eNB': [], # LTE + NR -> newLTE
        'SN_setup': [], # LTE -> LTE + NR => NR setup
        'SN_Rel': [], # LTE + NR -> LTE
        'SN_HO': [], # LTE + NR -> LTE + newNR  
        'RLF_II': [],
        'RLF_III': [],
        'SCG_RLF': [],
        'Add_SCell': [],
        }
    
    for i in range(len(df)):

        # Pass NR RRC packet. In NSA mode, LTE RRC packet include NR packet message.
        if NR_OTA(i) or LTE_SERV_INFO(i):
            continue

        try: lte_pci, lte_earfcn
        except: 
            lte_pci = df["PCI"].iloc[i]
            lte_earfcn = int(df["Freq"].iloc[i])

        others = ''
        t = df["Timestamp"].iloc[i]

        if df["rrcConnectionRelease"].iloc[i] == 1:
            D['Conn_Rel'].append(HO(start=t))
            nr_pci = 'O'

        if df["rrcConnectionRequest"].iloc[i] == 1:
            
            # Define end of rrcConnectionRequest to be rrcConnectionReconfigurationComplete or securityModeComplete.
            a = find_1st_after(i, 'rrcConnectionReconfigurationComplete',look_after=2)[0]
            b = find_1st_after(i, 'securityModeComplete',look_after=2)[0]
        
            if a is None and b is None: end = None
            elif a is None and b is not None: end = b
            elif a is not None and b is None: end = a 
            else: end = a if a > b else b
            
            _, idx = find_1st_after(i, 'ueCapabilityInformation',look_after=1)
            if idx is not None:
                sup_band = df['bandEUTRA'].iloc[idx]
                others += f' supported band: {sup_band}.' 

            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            trans = f'({lte_pci}, {lte_earfcn}) -> ({serv_cell}, {serv_freq})'
            
            # Check if caused by RLF III.
            a, idx = find_1st_before(i, 'rrcConnectionReestablishmentReject', look_before=1)
            if a is not None:
                others += ' After RLF III.'

            D['Conn_Req'].append(HO(start=t,end=end,trans=trans, others=others))

            nr_pci = 'O'
            lte_pci = serv_cell
            lte_earfcn = serv_freq
            
        if df["lte-rrc.t304"].iloc[i] == 1:
            
            end, _ = find_1st_after(i, 'rrcConnectionReconfigurationComplete')
            serv_cell, target_cell = df["PCI"].iloc[i], int(df['lte_targetPhysCellId'].iloc[i])
            serv_freq, target_freq = int(df["Freq"].iloc[i]), int(df['dl-CarrierFreq'].iloc[i])

            lte_pci = target_cell
            lte_earfcn = target_freq

            if df["SCellToAddMod-r10"].iloc[i] == 1:
                n =len(str(df["SCellIndex-r10.1"].iloc[i]).split('@'))
                others += f' Set up {n} SCell.'
            else:
                scells = []
            
            if serv_freq != target_freq:
                a,b = find_1st_before(i, "rrcConnectionReestablishmentRequest", 1)
                others += " Inter frequency HO."
                if a is not None:
                    others += " Near after RLF."
                
            if df["nr-rrc.t304"].iloc[i] == 1 and df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 2)
                    
                    if a is not None:

                        ho_type, ho = find_in_D_first_before(['RLF_II', 'RLF_III'], 2)
                        others += f' Near after RLF of trans: {ho.trans}.'

                    else:
                        
                        ho_type, _ = find_in_D_first_before(['MN_HO_to_eNB', 'SN_Rel'], 2)
                        if ho_type is not None:
                            others += f' Near after {ho_type}.'

                    ori_serv = nr_pci
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) | {ori_serv} -> {nr_pci}'
                    D['SN_setup'].append(HO(start=t, end=end, others=others, trans=trans))

                else:
                    
                    nr_pci = int(df['nr_physCellId'].iloc[i])
                    trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                    D['MN_HO'].append(HO(start=t, end=end, others=others, trans=trans))

            else:
                
                if serv_cell == target_cell and serv_freq == target_freq:

                    a, b = find_1st_before(i, "scgFailureInformationNR-r15")
                    if a is not None:
                        others += " Caused by scg-failure."
                    
                    orig_serv = nr_pci
                    nr_pci = 'O'
                    trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                    D['SN_Rel'].append(HO(start=t, end=end, others=others, trans=trans))
                    
                else:

                    a, _ = find_1st_before(i,"rrcConnectionSetup",3)
                    if a is not None:
                        others += ' Near After connection setup.'
                    if nr_pci == 'O':
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {nr_pci}'
                        D['LTE_HO'].append(HO(start=t, end=end, others=others, trans=trans))
                    else:
                        orig_serv = nr_pci
                        nr_pci = 'O'
                        trans = f'({serv_cell}, {serv_freq}) -> ({target_cell}, {target_freq}) | {orig_serv} -> {nr_pci}'
                        D['MN_HO_to_eNB'].append(HO(start=t, end=end, others=others, trans=trans))


        if df["nr-rrc.t304"].iloc[i] == 1 and not df["dualConnectivityPHR: setup (1)"].iloc[i] == 1:

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
        
            serv_cell, serv_freq = df["PCI"].iloc[i], int(df["Freq"].iloc[i])
            orig_serv = nr_pci
            nr_pci = int(df['nr_physCellId'].iloc[i])
            trans = f'({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
            D['SN_HO'].append(HO(start=t,end=end,trans=trans))


        if df["rrcConnectionReestablishmentRequest"].iloc[i] == 1:

            end1, _ = find_1st_after(i, 'rrcConnectionReestablishmentComplete', look_after=1)
            b, _ = find_1st_after(i, 'rrcConnectionReestablishmentReject', look_after=1)
            end2, _ = find_1st_after(i, 'securityModeComplete',look_after=3)

            others += ' ' + df["reestablishmentCause"].iloc[i] + '.'
            scells = []

            c, _ = find_1st_before(i, 'scgFailureInformationNR-r15', 1)
            if c != None:
                others  += ' caused by scgfailure.'
                
            serv_cell, rlf_cell = df["PCI"].iloc[i], int(df['physCellId.3'].iloc[i])
            serv_freq = int(df['Freq'].iloc[i])
            
            # Type II & Type III
            if end1 is not None: 

                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])
                trans = f'({rlf_cell}, {rlf_freq}) -> ({serv_cell}, {serv_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_II'].append(HO(start=t,end=end1,others=others,trans=trans))

                lte_pci = serv_cell
                lte_earfcn = serv_freq

            elif b is not None and end2 is not None:
                
                orig_serv = nr_pci
                nr_pci = 'O'
                _, idx = find_1st_before_with_special_value(i, 'PCI', rlf_cell, look_before=10)
                rlf_freq = int(df['Freq'].iloc[idx])

                _, idx = find_1st_after(i, "rrcConnectionRequest", 2)
                recon_cell, recon_freq = df['PCI'].iloc[idx], int(float(df['Freq'].iloc[idx]))
                
                trans = f'({rlf_cell}, {rlf_freq}) -> ({recon_cell}, {recon_freq}) | {orig_serv} -> {nr_pci}'
                D['RLF_III'].append(HO(start=t,end=end2,others=others,trans=trans)) 

                # lte_pci, lte_earfcn will be updated in rrcConnectionRequest.     
                
            else:

                others+=' No end.'
                D['RLF_II'].append(HO(start=t,others=others))
                print('No end for RLF')

        if df["scgFailureInformationNR-r15"].iloc[i] == 1:

            others += ' ' + df["failureType-r15"].iloc[i] + '.'
            a, idx1 = find_1st_after(i, "rrcConnectionReestablishmentRequest", look_after=1)
            b, idx2 = find_1st_after(i, "lte-rrc.t304", look_after=10)

            if a is not None:

                end1, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentComplete', look_after=1)
                b, _ = find_1st_after(idx1, 'rrcConnectionReestablishmentReject', look_after=1)
                end2 = find_1st_after(idx1, 'securityModeComplete',look_after=3)[0]

                others += ' Result in rrcReestablishment.'
                    
                # Type II & Type III Result
                if end1 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end1,others=others))
                elif b is not None and end2 is not None: 
                    D['SCG_RLF'].append(HO(start=t,end=end2,others=others))
                else:
                    others += ' No end.'
                    D['SCG_RLF'].append(HO(start=t,others=others))
                    print('No end for scg failure result in rrcReestablishment.')

            elif b is not None:

                end, _ = find_1st_after(idx2, 'rrcConnectionReconfigurationComplete')
                serv_cell, target_cell = df["PCI"].iloc[idx2], df['lte_targetPhysCellId'].iloc[idx2]
                serv_freq, target_freq = int(df["Freq"].iloc[idx2]), df['dl-CarrierFreq'].iloc[idx2]
                # We do not change nr_pci here. Instead, we will change it at gNB_Rel event.
                trans = f'({serv_cell}, {serv_freq}) | {nr_pci} -> O'
                
                if serv_cell == target_cell and serv_freq == target_freq:
                    others += ' Result in gNB release.'
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))
                else:
                    others += ' Result in MN HO to eNB.'
                    D['SCG_RLF'].append(HO(start=t,end=end,others=others,trans=trans))                  

            else:

                print('No end for scg failure.')
                others += ' No end.'
                D['SCG_RLF'].append(HO(start=t,others=others))
        
        if df['SCellToAddMod-r10'].iloc[i] == 1 and df['physCellId-r10'].iloc[i] != 'nr or cqi report':

            others = ''
            pcis = str(df["physCellId-r10"].iloc[i]).split('@')
            freqs = str(df["dl-CarrierFreq-r10"].iloc[i]).split('@')
            orig_scells = scells
            scells = [(int(float(pci)), int(float(freq))) for pci, freq in zip(pcis, freqs)]

            others += f' Set up {len(scells)} SCell.'
            trans = f'{orig_scells} -> {scells}'

            end, _ = find_1st_after(i,'rrcConnectionReconfigurationComplete')
            
            a, _ = find_1st_before(i, "rrcConnectionReestablishmentRequest", 3)
            if a is not None:
                others += ' Near after RLF.'

            a = find_in_D_exact(['LTE_HO', 'MN_HO', 'MN_HO_to_eNB', 'SN_setup', 'SN_Rel'])
            if a is not None:
                others += f' With {a}.'

            D['Add_SCell'].append(HO(start=t,end=end,others=others, trans=trans))
    
    return D


### PCAP to CSV

In [ ]:
# Run only a file.
filename = ''
subprocess.run(["python3", "/home/wmnlab/github_code/ntu-experiments/sheng-ru/post_processing/pcap_to_csv.py", f"{filename}"])
pass

In [ ]:
# Run within a directory.
dirname = "/home/wmnlab/Process/"
subprocess.run(["python3", "/home/wmnlab/github_code/ntu-experiments/sheng-ru/post_processing/pcap_to_csv.py", f"{dirname}"])
pass

### Loss Latency Calculate

In [20]:
# Some useful settings.

# Read time sync file
time_sync_file = '/home/wmnlab/Process/time_sync_lpt3.json'
with open(time_sync_file, 'r') as f:
    time_off_dict = json.load(f)
    
TAG = "000425d401df5e76"
pattern = r'(\d{4}-\d{2}-\d{2}_\d{2}-\d{2}-\d{2})'
PKG = namedtuple('PKG', ['timestamp', 'seq','lat'], defaults=[None, 0, 0])
excessive_value = 0.1

def find_missing_numbers(input_list):
    min_num = min(input_list)
    max_num = max(input_list)
    all_numbers = set(range(min_num, max_num + 1))
    input_set = set(input_list)
    missing_numbers = list(all_numbers - input_set)
    return sorted(missing_numbers)

In [21]:
# Calculate "Client" pcap DL Latency, 
dirname ='/home/wmnlab/Process/' 
file1 = 'client_pcap_BL_qc00_3270_3271_2023-10-24_16-19-53_sock_pcap.csv'
file2 = 'client_pcap_BL_qc03_3276_3277_2023-10-24_16-19-53_sock_pcap.csv'

if not file1.endswith('_pcap.csv') or not file2.endswith('_pcap.csv'):
    raise

file1 = os.path.join(dirname, file1)
file2 = os.path.join(dirname, file2)
print(file1,file2,sep='\n')

match = re.search(pattern, file1)
if match:
    datetime_str = match.group(1)
    filetime = pd.to_datetime(datetime_str, format='%Y-%m-%d_%H-%M-%S')
else:
    print("File Name Error!")
    raise

measure_times = [pd.to_datetime(k, format='%Y-%m-%d %H:%M:%S.%f') for k in time_off_dict.keys()]
time_diffs = [abs(t-filetime) for t in measure_times]
ind, value = min(enumerate(time_diffs), key=lambda x: x[1])
off = list(time_off_dict.values())[ind]

for id, file in enumerate([file1, file2]):
    
    df = pd.read_csv(file, sep='@')
    cond = df['ip.src'] == '140.112.20.183'
    df = df[cond] # Take only the DL package.
    df['frame.time'] = pd.to_datetime(df['frame.time'])
    df['frame.time'] = df['frame.time'].dt.tz_localize(None) 

    pkgs = []

    for i in range(len(df)):
        
        arrive_time = df['frame.time'].iloc[i] + dt.timedelta(seconds=off)
        payload = df['udp.payload'].iloc[i]

        t = payload[16:24]  # YY:MM:DD hh:mm:ss
        usec = payload[24:32] # microseconds
        seq = payload[32:40] # Package Sequence

        send_time = dt.datetime.utcfromtimestamp(int(t, 16)) + dt.timedelta(hours=8, microseconds=int(usec, 16))
        lat = (arrive_time-send_time).total_seconds()
        
        pkgs.append(PKG(timestamp= send_time, seq=int(seq, 16), lat=lat))

    if id == 0:
        sorted_pkgs1 = sorted(pkgs, key=lambda pkg: pkg.seq)
        timestamps1 = [pkg.timestamp for pkg in sorted_pkgs1]
        lats1 = [pkg.lat for pkg in sorted_pkgs1]
        seqs1 = [pkg.seq for pkg in sorted_pkgs1]
        exc_lat_pkgs1 = [pkg for pkg in sorted_pkgs1 if pkg.lat > excessive_value]
        loss_pkg_seqs1 = find_missing_numbers(seqs1)
        PLR1 = round(len(loss_pkg_seqs1)/len(sorted_pkgs1)*100, 2) # %
        ELR1 = round(len(exc_lat_pkgs1)/len(sorted_pkgs1)*100, 2) # %
        print(f'PLR1: {PLR1}%, ELR1: {ELR1}%')
    elif id == 1:
        sorted_pkgs2 = sorted(pkgs, key=lambda pkg: pkg.seq)
        timestamps2 = [pkg.timestamp for pkg in sorted_pkgs2]
        lats2 = [pkg.lat for pkg in sorted_pkgs2]
        seqs2 = [pkg.seq for pkg in sorted_pkgs2]
        exc_lat_pkgs2 = [pkg for pkg in sorted_pkgs2 if pkg.lat > excessive_value]
        loss_pkg_seqs2 = find_missing_numbers(seqs2)
        PLR2 = round(len(loss_pkg_seqs2)/len(sorted_pkgs2)*100, 2) # %
        ELR2 = round(len(exc_lat_pkgs2)/len(sorted_pkgs2)*100, 2) # %
        print(f'PLR2: {PLR2}%, ELR2: {ELR2}%')

/home/wmnlab/Process/client_pcap_BL_qc00_3270_3271_2023-10-24_16-19-53_sock_pcap.csv
/home/wmnlab/Process/client_pcap_BL_qc03_3276_3277_2023-10-24_16-19-53_sock_pcap.csv
PLR1: 0.42%, ELR1: 0.65%
PLR2: 0.0%, ELR2: 0.56%


In [27]:
# Calculate Dual Radio Performance
seq_start = seqs1[0] if seqs1[0] <= seqs2[0] else seqs2[0]
seq_end = seqs1[-1] if seqs1[-1] <= seqs2[-1] else seqs2[-1]

dual_loss_seqs = set(loss_pkg_seqs1) & set(loss_pkg_seqs2)
dual_loss_seqs = sorted(list(dual_loss_seqs))

excl_seqs1 = [pkg.seq for pkg in exc_lat_pkgs1]
excl_seqs2 = [pkg.seq for pkg in exc_lat_pkgs2]
dual_excl_seqs = set(excl_seqs1) & set(excl_seqs2)
dual_excl_seqs = sorted(list(dual_excl_seqs))

DPLR = len(dual_loss_seqs)/(seq_end-seq_start+1)
DELR = len(dual_excl_seqs)/(seq_end-seq_start+1)
print(f'Dual PLR: {DPLR}, Dual ELR: {DELR}')

Dual PLR: 0.0, Dual ELR: 0.0


In [ ]:
# Plot
# plt.plot(lats1)
# plt.show()

### Handover

In [ ]:
# mi2log -> txt

In [ ]:
parse_mi_ho